# SOTA Hotpot model get predictions

In [ ]:
!git clone https://github.com/JD-AI-Research-Silicon-Valley/SAE.git


Cloning into 'SAE'...
remote: Enumerating objects: 344, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 344 (delta 32), reused 74 (delta 25), pack-reused 256
Receiving objects: 100% (344/344), 72.38 MiB | 44.84 MiB/s, done.
Resolving deltas: 100% (111/111), done.


In [ ]:
!pip install --upgrade --no-cache-dir gdown


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=bb5e55eb218cf6d228c124e55e3e3698bab550d5c401b868240d44fe93fe84f3
  Stored in directory: /tmp/pip-ephem-wheel-cache-260v_lsu/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [ ]:
!gdown --id 1Eqgi0SYB9XRHkuMyjFpeYI_MWZ7GCNTt


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Eqgi0SYB9XRHkuMyjFpeYI_MWZ7GCNTt
To: /content/models.zip
100% 3.90G/3.90G [00:18<00:00, 215MB/s]


In [ ]:
!pip install fastparquet


     |████████████████████████████████| 1.4 MB 5.1 MB/s 
     |████████████████████████████████| 134 kB 76.9 MB/s 
     |████████████████████████████████| 1.6 MB 52.5 MB/s 


In [ ]:
!unzip -q models.zip

In [ ]:
!pip install -r SAE/requirements.txt

     |████████████████████████████████| 132 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 40.7 MB/s 
     |████████████████████████████████| 8.6 MB 56.7 MB/s 
     |████████████████████████████████| 79 kB 10.8 MB/s 
     |████████████████████████████████| 138 kB 73.5 MB/s 
     |████████████████████████████████| 127 kB 76.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
!mv SAE/* .


In [ ]:
#!rm -r output
!mkdir output

Prep data as in the experiment

In [ ]:
import json

with open('hotpot_dev_distractor_v1.json') as f:
    data = json.load(f)

In [ ]:
import random
import copy
def remove_distractors_remove_key_sent(example, n_distractors = 0):
    all_pars = [example['context'][i][0] for i in range(len(example['context']))]
    gold_pars = list(set([example['supporting_facts'][i][0] for i in range(len(example['supporting_facts']))]))
    distractor_pars = list(set(all_pars) - set(gold_pars))
    # get indices to keep from disractors
    if len(distractor_pars) == 0 or n_distractors == 0:
        distract_indices = []
    else:
        distract_indices = random.sample(range(len(distractor_pars)), n_distractors)
    distractor_pars = [distractor_pars[idx] for idx in distract_indices]
    keep_pars = gold_pars + distractor_pars
    keep_pars_indices = [all_pars.index(keep_par) for keep_par in keep_pars]
    example['context'] = [example['context'][i] for i in range(len(example['context'])) if example['context'][i][0] in keep_pars]

    # removed all distractors, now we remove a key sentence
    rand_sup_idx =random.sample(list(range(len(example['supporting_facts']))), 1)[0]
    title_to_index = {}
    for i in range(len(example['context'])):
        title_to_index[example['context'][i][0]] = i
    par_to_remove = title_to_index[example['supporting_facts'][rand_sup_idx][0]]
    sent_to_remove = example['supporting_facts'][rand_sup_idx][1]
    context = copy.deepcopy(example['context'])
    able_to_remove = False
    times_tried = 0
    while not able_to_remove:
        if times_tried >4:
            print("giving up")
            break
        try:
            sent = context[par_to_remove][1][sent_to_remove]
            if example['answer'] in sent:
                raise Exception('error')
            context[par_to_remove][1].pop(sent_to_remove)
            able_to_remove = True
        except:
            times_tried += 1
            print("failed trying again")
            rand_sup_idx =random.sample(list(range(len(example['supporting_facts']))), 1)[0]
            title_to_index = {}
            for i in range(len(example['context'])):
                title_to_index[example['context'][i][0]] = i
            par_to_remove = title_to_index[example['supporting_facts'][rand_sup_idx][0]]
            sent_to_remove = example['supporting_facts'][rand_sup_idx][1]
            context = copy.deepcopy(example['context'])
    example['context'] = context
    return example

In [ ]:
data_fixed = []
for i in range(len(data)):
    ex_fixed = remove_distractors_remove_key_sent(data[i])
    data_fixed.append(ex_fixed)
with open('dev_hotpot_nodistractors.json', 'w') as json_file:
    json.dump(data_fixed, json_file)

In [ ]:
!python main.py dev_hotpot_nodistractors.json


In [ ]:
!zip -r outputs_val.zip output


In [ ]:
import json
with open('hotpot_dev_distractor_v1.json') as f:
  data = json.load(f)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}


In [ ]:
while True:
    a= 1

In [ ]:
data = data[:20]

In [ ]:
import pickle
with open('output/embeddings_data.p', 'rb') as handle:
    embedds = pickle.load(handle)

In [ ]:
with open('test_hot.json', 'w') as json_file:
  json.dump(data, json_file)

In [ ]:
!pip install ujson

In [ ]:
!python hotpot_evaluate_v1.py output/predictions_ans.json hotpot_dev_distractor_v1.json

{'em': 0.6203916272788657, 'f1': 0.7511464914313525, 'prec': 0.7778640767784575, 'recall': 0.7625362560389581, 'sp_em': 0.0, 'sp_f1': 0.0, 'sp_prec': 0.0, 'sp_recall': 0.0, 'joint_em': 0.0, 'joint_f1': 0.0, 'joint_prec': 0.0, 'joint_recall': 0.0}


dev set {'em': 0.6203916272788657, 'f1': 0.7511464914313525, 'prec': 0.7778640767784575, 'recall': 0.7625362560389581, 'sp_em': 0.0, 'sp_f1': 0.0, 'sp_prec': 0.0, 'sp_recall': 0.0, 'joint_em': 0.0, 'joint_f1': 0.0, 'joint_prec': 0.0, 'joint_recall': 0.0}
